# Decision Trees
Decision Trees are ML algorithms that progressively divide data sets into smaller data groups based on a descriptive feature, until they reach sets that are small enough to be described by some label.

### Main DT Algorithms
####  1. CHAID - Chi-squared Automatic Interaction Detection
When building **classification trees**, CHAID relies on chi-squared tests to find the best split at each step. In other words, it chooses the independent variable that has the strongest interaction with the dependent variable. For **regression trees**, CHAID relies on F-tests to calculate the difference between two population means.

#### 2. CART - Classification And Regression Trees
In the case of **Classification Trees**, CART algorithm uses a metric called **Gini Impurity** to create decision points for classification tasks. Gini Impurity gives an idea of how fine a split is. In the case of **Regression Trees**, CART algorithm looks for splits that minimize the **Least Square Deviation (LSD)**, choosing the partitions that minimize the result over all possible options. The LSD (sometimes referred as “variance reduction”) metric minimizes the sum of the squared distances (or deviations) between the observed values and the predicted values.

#### 3. ID3 - Iterative Dichotomiser 3
It is mostly used for classification tasks. ID3 splits data attributes (dichotomizes) to find the most dominant features, performing this process iteratively to select the DT nodes in a top-down approach. For the splitting process, ID3 uses the **Information Gain** metric to select the most useful attributes for classification. Information Gain is directly linked to the concept of **Entropy**, which is the measure of the amount of uncertainty or randomness in the data.

#### 4. C 4.5 
It is successor of ID3. C4.5 can handle both continuous and categorical data, making it suitable to generate Regression and Classification Trees. Additionally, it can deal with missing values by ignoring instances that include non-existing data. Unlike ID3, C4.5 uses **Gain Ratio** for its splitting process. Gain Ratio is a modification of the Information Gain concept that reduces the bias on DTs. Another capability of C4.5 is that it can prune DTs.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### Here we will implement CART algorithm for Regression task
#### Important points
1. The representation of the CART model is a binary tree.
2. For regression, The cost function that is minimized to choose split points is the sum squared error(MSE) across all training samples that fall within the node.
3. For classification, The Gini cost function is used which provides an indication of how pure the nodes are, where node purity refers to how mixed the training data assigned to each node is.

### 1. MSE Criterion
* Name of the cost function used to evaluate splits in the dataset.
* Performs only binary splits
* Higher the value of mse higher the error.

**Steps to calculate cost**:
1. Calculate Mean squared Error of y for each node.
2. Calculate Gini for split using weighted mse of each node of that split


$$ Cost\;for\;split = \sum ( MSE\;for\;node\;y\;values * \frac{node\;size}{total\;size}) $$



### 2. Terminal Node
When to decide to stop growing the tree

**Maximum Tree Depth** : This is the maximum number of nodes from the root node of the tree. Once a maximum depth of the tree is met, we must stop splitting adding new nodes. Deeper trees are more complex and are more likely to overfit the training data.

**Minimum Node Records** :  This is the minimum number of training patterns that a given node is responsible for. Once at or below this minimum, we must stop splitting and adding new nodes. Nodes that account for too few training patterns are expected to be too specific and are likely to overfit the training data.

There is one more condition. It is possible to choose a split in which all rows belong to one group. In this case, we will be unable to continue splitting and adding child nodes as we will have no records to split on one side or another.

Now we have some ideas of when to stop growing the tree. When we do stop growing at a given point, that node is called a terminal node and is used to make a final prediction.

This is done by taking the group of rows assigned to that node and selecting the most common class value in the group. This will be used to make predictions.

In [2]:
class DecisionTreeRegressor():
    def __init__(self,criterion='mse'):
        """
        Args :
        criterion : mse : mean squared error, mae : mean absoulute error, std : standard deviation
        """
        self.root = None
        self.max_depth = 0
        self.cost = { 'mse':self.__mse,'std':self.__std,'mae':self.__mae }[criterion]
        self.X = None
        self.y = None
        
    def __std(self,y):
        squared_error = (y-y.mean())**2
        return np.sqrt(  np.sum(squared_error)/len(y)  )
    
    def __mse(self,y):
        squared_error = (y-y.mean())**2
        return np.sum( squared_error/len(y) )
    
    def __mae(self,y): return np.sum(abs(y-y.mean())/len(y))
    
    def computeCost(self,groups, y):
        n_instances = len(groups[0])+len(groups[1])  # count of all samples
        weighted_cost = 0.0 # sum weighted Gini index for each group
        for indexes in groups:
            size = len(indexes)
            # avoid divide by zero
            if size == 0: continue
            weighted_cost +=  self.cost(y[indexes]) * (size/n_instances)
        return weighted_cost
    
    def get_split(self,X,y):
        b_index, b_value, b_cost, b_groups = float('inf'), float('inf'), float('inf'), None
        for col_ind in range(X.shape[1]): #no of features
            for val in np.unique(X[:,col_ind]): #for each unique value in each of the features

                #left_index indexes lower than val for feature, right_index indexes greater that val for feature
                left_index = np.reshape( np.argwhere(X[:,col_ind]<val), (-1,) )
                right_index = np.reshape( np.argwhere(X[:,col_ind]>=val), (-1,) )
                
                #find gini index
                cost = self.computeCost((left_index,right_index), y)
                if cost < b_cost:
                    b_index, b_value, b_cost, b_groups = col_ind, val, cost, (left_index, right_index)
        return {'index':b_index, 'value':b_value, 'groups':b_groups}
    
    def to_terminal(self,y): return y.mean()
    
    def split(self,node, X, y, max_depth, min_samples_split, depth):
        self.max_depth = max(depth,self.max_depth)
        left, right = node.pop('groups')
        
        # check for a no split
        if len(left)==0 or len(right)==0:
            node['left'] = node['right'] = self.to_terminal(y[np.append(left,right)])
            return
        
        # check for max depth
        if depth >= max_depth:
            node['left'], node['right'] = self.to_terminal(y[left]), self.to_terminal(y[right])
            return
        
        # process left child
        if len(left) <= min_samples_split:
            node['left'] = self.to_terminal(y[left])
        else:
            node['left'] = self.get_split(X[left],y[left])
            self.split(node['left'], X[left], y[left], max_depth, min_samples_split, depth+1)
        
        # process right child
        if len(right) <= min_samples_split:
            node['right'] = self.to_terminal(y[right])
        else:
            node['right'] = self.get_split(X[right],y[right])
            self.split(node['right'],X[right],y[right], max_depth, min_samples_split, depth+1)

    def fit(self, X, y, max_depth=None, min_samples_split=2):
        self.X, self.y, max_depth = X, y, float('inf') if max_depth==None else max_depth
        self.root = self.get_split(X,y)
        self.split(self.root, X, y, max_depth, min_samples_split,1)
        
    def predict_row(self,row,node):
        if row[node['index']] < node['value']:
            if isinstance(node['left'], dict): return self.predict_row(row,node['left'])
            else: return node['left']
        else:
            if isinstance(node['right'], dict): return self.predict_row(row,node['right'])
            else: return node['right']
    
    def predict(self,rows): return np.array( [self.predict_row(row,self.root) for row in rows] )
    
    def score(self,X,y):
        "Coefficient of Determination: r2"
        y_pred = self.predict(X)
        return 1-( np.sum( (y-y_pred)**2 )/np.sum( (y-y.mean())**2 ) )
    
    @property
    def depth(self): return self.max_depth
    
    @property
    def tree_(self): return self.root

In [3]:
data = pd.read_csv('data.csv')
data = data.sample(frac=1)
data.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
483,2.81838,0.0,18.10,0,0.532,5.762,40.3,4.0983,24,666,20.2,392.92,10.42,21.8
458,7.75223,0.0,18.10,0,0.713,6.301,83.7,2.7831,24,666,20.2,272.21,16.23,14.9
391,5.29305,0.0,18.10,0,0.700,6.051,82.5,2.1678,24,666,20.2,378.38,18.76,23.2
129,0.88125,0.0,21.89,0,0.624,5.637,94.7,1.9799,4,437,21.2,396.90,18.34,14.3
297,0.14103,0.0,13.92,0,0.437,5.790,58.0,6.3200,4,289,16.0,396.90,15.84,20.3


In [4]:
def train_test_split(X,y,test_size=0.3):
    indexes = np.random.choice( [False,True], len(y), p=[test_size,1-test_size])
    return X[indexes],X[~indexes],y[indexes],y[~indexes]

X = data.drop('medv',axis=1).values
y = data.medv.values

X_train,X_val,Y_train,Y_val = train_test_split(X,y)
X_train.shape,X_val.shape,Y_train.shape,Y_val.shape

((359, 13), (147, 13), (359,), (147,))

In [5]:
dt = DecisionTreeRegressor(criterion='mse')
dt.fit(X_train,Y_train)
dt.score(X_val,Y_val),dt.depth

(0.6075823746728266, 18)

In [6]:
np.unique(dt.predict(X_val),return_counts=True)

(array([ 5.  ,  6.3 ,  7.  ,  7.2 ,  7.45,  8.3 , 10.9 , 11.05, 11.4 ,
        11.7 , 12.15, 13.65, 14.  , 14.2 , 14.35, 14.8 , 15.3 , 15.4 ,
        15.5 , 15.7 , 16.6 , 16.7 , 17.05, 17.5 , 17.8 , 18.15, 18.25,
        18.5 , 18.6 , 18.7 , 19.25, 19.3 , 19.5 , 19.85, 20.25, 20.55,
        20.75, 20.9 , 21.15, 21.2 , 21.45, 21.7 , 21.75, 22.  , 22.2 ,
        22.25, 22.5 , 22.6 , 22.8 , 22.85, 22.9 , 23.2 , 23.4 , 23.7 ,
        23.8 , 23.95, 24.05, 24.55, 24.8 , 26.4 , 27.35, 28.55, 28.6 ,
        28.85, 29.1 , 30.8 , 31.05, 31.2 , 31.5 , 32.  , 32.25, 33.2 ,
        34.9 , 35.  , 36.2 , 36.45, 43.9 , 46.55, 50.  ]),
 array([1, 1, 1, 1, 1, 2, 2, 1, 1, 4, 5, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1,
        1, 1, 3, 1, 1, 4, 4, 6, 1, 1, 1, 1, 1, 2, 1, 1, 1, 4, 2, 4, 1, 4,
        2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 7, 2, 2, 1, 2, 2, 1, 1, 2, 5, 1, 2,
        1, 1, 2, 2, 2, 1, 3, 1, 2, 2, 1, 2, 5]))